# Alternating least squares for Canonical Polyadic (CP) Decomposition

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

The function `cp_als` computes an estimate of the best rank-R CP model of a tensor X using the well-known alternating least-squares algorithm (see, e.g., Kolda and Bader, SIAM Review, 2009, for more information). The input X can be almost any type of tensor including a `tensor`, `sptensor`, `ktensor`, or `ttensor`. The output CP model is a `ktensor`.

In [ ]:
import os
import sys
import pyttb as ttb
import numpy as np

## Load some data

### TODO: Choose whether to include sample datasets from MATLAB TTB
```
We use the well-known *amino acids data set* from Andersson and Bro. It contains fluorescence measurements of 5 samples containing 3 amino acids: Tryptophan, Tyrosine, and Phenylalanine. Each amino acid corresponds to a rank-one component. The tensor is of size 5 x 51 x 201 from 5 samples, 51 excitations, and 201 emissions. Further details can be found here: http://www.models.life.ku.dk/Amino_Acid_fluo. Please cite the following paper for this data: Rasmus Bro, PARAFAC: Tutorial and applications, Chemometrics and Intelligent Laboratory Systems, 1997, 38, 149-171. This dataset can be found in the `doc` directory.

We will just use random data for now.
```

In [ ]:
# Pick the size and rank
R = 3
np.random.seed(0)  # Set seed for reproducibility
X = ttb.tensor(np.random.rand(6, 8, 10), shape=(6, 8, 10))

## Basic call to the method, specifying the data tensor and its rank

This uses a *random* initial guess. At each iteration, it reports the *fit* `f` which is defined as 
```
f = 1 - ( X.norm()**2 + M.norm()**2 - 2*<X,M> ) / X.norm()
``` 
and is loosely the proportion of the data described by the CP model, i.e., a fit of 1 is perfect.

In [ ]:
# Compute a solution with final ktensor stored in M1
np.random.seed(0)  # Set seed for reproducibility
short_tutorial = 10  # Cut off solve early for demo
M1 = ttb.cp_als(X, R, maxiters=short_tutorial)

Since we set only a single output, `M1` is actually a *tuple* containing:
1. `M1[0]`: the solution as a `ktensor`. 
2. `M1[1]`: the initial guess as a `ktensor` that was generated at runtime since no initial guess was provided. 
3. `M1[2]`: a dictionary containing runtime information with keys:
    * `params`: parameters used by `cp_als`
    * `iters`: number of iterations performed
    * `normresidual`: the norm of the residual `X.norm()**2 + M.norm()**2 - 2*<X,M>`
    * `fit`: the fit `f` described above

In [ ]:
print(f"M1[2]['params']: {M1[2]['params']}")
print(f"M1[2]['iters']: {M1[2]['iters']}")
print(f"M1[2]['normresidual']: {M1[2]['normresidual']}")
print(f"M1[2]['fit']: {M1[2]['fit']}")

## Run again with a different initial guess, output the initial guess.

In [ ]:
np.random.seed(1)  # Set seed for reproducibility
M2bad, Minit, _ = ttb.cp_als(X, R, maxiters=short_tutorial)

## Increase the maximium number of iterations
Note that the previous run kicked out at only 10 iterations, before reaching the specified convegence tolerance. Let's increase the maximum number of iterations and try again, using the same initial guess.

In [ ]:
less_short_tutorial = 10 * short_tutorial
M2 = ttb.cp_als(X, R, maxiters=less_short_tutorial, init=Minit)

## Compare the two solutions
Use the `ktensor` `score()` member function to compare the two solutions. A score of 1 indicates a perfect match.

In [ ]:
M1_ktns = M1[0]
M2_ktns = M2[0]
score = M1_ktns.score(M2_ktns)

Here, `score()` returned a tuple `score` with the score as the first element:

In [ ]:
score[0]

See the `ktensor` documentation for more information about the return values of `score()`.

# Rerun with same initial guess
Using the same initial guess (and all other parameters) gives the exact same solution.

In [ ]:
M2alt = ttb.cp_als(X, R, maxiters=less_short_tutorial, init=Minit)
M2alt_ktns = M2alt[0]
score = M2_ktns.score(M2alt_ktns)  # Score of 1 indicates the same solution
print(f"Score: {score[0]}.")

## Changing the output frequency
Using the `printitn` option to change the output frequency.

In [ ]:
M2alt2 = ttb.cp_als(X, R, maxiters=less_short_tutorial, init=Minit, printitn=20)

## Suppress all output
Set `printitn` to zero to suppress all output.

In [ ]:
# TODO this will pass when issue #235 is resolved
M2alt2 = ttb.cp_als(X, R, printitn=0)  # No output

## Use HOSVD initial guess
Use the `'nvecs'` option to use the leading mode-n singular vectors as the initial guess.

In [ ]:
M3 = ttb.cp_als(X, R, init="nvecs", printitn=20)
s = M2[0].score(M3[0])
print(f"score(M2,M3) = {s[0]}")

# Change the order of the dimensions in CP

In [ ]:
M4, _, info = ttb.cp_als(X, 3, dimorder=[1, 2, 0], init="nvecs", printitn=20)
s = M2[0].score(M4)
print(f"score(M2,M4) = {s[0]}")

In the last example, we also collected the third output argument `info` which has runtime information in it. The field `info['iters']` has the total number of iterations. The field `info.['params']` has the information used to run the method. Unless the initialization method is 'random', passing the parameters back to the method will yield the exact same results.

In [ ]:
# TODO this will pass when issue #236 is resolved
M4alt, _, info = ttb.cp_als(X, 3, **info["params"])
s = M4alt.score(M4)
print(f"score(M4alt,M4) = {s[0]}")

## Change the tolerance
It's also possible to loosen or tighten the tolerance on the change in the fit. You may need to increase the number of iterations for it to converge.

In [ ]:
M5 = ttb.cp_als(X, 3, init="nvecs", stoptol=1e-12, printitn=100)

## Control sign ambiguity of factor matrices
The default behavior of `cp_als` is to make a call to `fixsigns()` to fix the sign ambiguity of the factor matrices. You can turn off this behavior by passing the `fixsigns` parameter value of `False` when calling `cp_als`.

In [ ]:
X = ttb.ktensor(
    factor_matrices=[
        np.array([[1.0, 1.0], [1.0, -10.0]]),
        np.array([[1.0, 1.0], [1.0, -10.0]]),
    ],
    weights=np.array([1.0, 1.0]),
)
M1 = ttb.cp_als(X, 2, printitn=1, init=ttb.ktensor(X.factor_matrices))
print(M1[0])  # default behavior, fixsigns called
M2 = ttb.cp_als(X, 2, printitn=1, init=ttb.ktensor(X.factor_matrices), fixsigns=False)
print(M2[0])  # fixsigns not called

## Recommendations
* Run multiple times with different guesses and select the solution with the best fit.
* Try different ranks and choose the solution that is the best descriptor for your data based on the combination of the fit and the interpretaton of the factors, e.g., by visualizing the results.